<a href="https://colab.research.google.com/github/DanielCastilloRdz/Codigo_machine_learning/blob/main/modelos_supervisados%5B1%5D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import warnings
import numpy as np
from sklearn.model_selection import LeaveOneOut, GridSearchCV, cross_validate, cross_val_predict
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression,Ridge,Lasso,PoissonRegressor,GammaRegressor
from sklearn.tree import DecisionTreeClassifier,DecisionTreeRegressor
from sklearn.neighbors import KNeighborsClassifier,KNeighborsRegressor
from sklearn.svm import SVC,SVR
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier,AdaBoostRegressor,GradientBoostingRegressor,RandomForestRegressor
from sklearn.neural_network import MLPClassifier,MLPRegressor
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score,mean_squared_error, r2_score

warnings.filterwarnings("ignore")

def verificar_missing(dataframe):
    """
    Función para verificar valores faltantes en un DataFrame.

    Args:
        dataframe (pd.DataFrame): El DataFrame a analizar.

    Returns:
        pd.DataFrame: Un resumen con el número y porcentaje de valores faltantes por columna.
    """
    total_missing = dataframe.isnull().sum()
    porcentaje_missing = 100 * total_missing / len(dataframe)
    missing_df = pd.DataFrame({
        'Total de Missings': total_missing,
        'Porcentaje (%)': porcentaje_missing
    })
    missing_df = missing_df[missing_df['Total de Missings'] > 0].sort_values(by='Total de Missings', ascending=False)

    if missing_df.empty:
        print(" No hay valores faltantes en el DataFrame.")
    else:
        print("Hay valores faltantes:")

    return missing_df


In [ ]:
insurance = pd.read_csv("insurance_claims.csv",header=0,keep_default_na=False,na_values=[''])
verificar_missing(insurance)

Hay valores faltantes:


,Total de Missings,Porcentaje (%)
_c39,1000,100.0


In [ ]:

columnas_modelos = ['incident_date','insured_sex', 'incident_type', 'collision_type', 'incident_severity','property_damage', 'police_report_available',
     'insured_education_level','months_as_customer', 'age', 'policy_deductable',
     'policy_annual_premium', 'umbrella_limit', 'capital-gains', 'capital-loss',
     'incident_hour_of_the_day','number_of_vehicles_involved', 'bodily_injuries',
     'witnesses', 'auto_year', 'total_claim_amount', 'fraud_reported']

categorical_columns = ['insured_sex','insured_education_level','incident_type', 'collision_type', 'incident_severity','property_damage','police_report_available']
insurance  = insurance[columnas_modelos]
insurance = insurance.sort_values(by='incident_date').reset_index(drop=True)
#insurance[categorical_columns] = insurance[categorical_columns].astype('category')

In [ ]:
insurance["fraud_reported"].value_counts()

fraud_reported
N    753
Y    247
Name: count, dtype: int64

In [ ]:
columnas = insurance.columns
X = insurance[columnas[1:20]]
y_fraud = insurance[columnas[-1]]
y_claim = insurance[columnas[-2]]
print(columnas)

Index(['incident_date', 'insured_sex', 'incident_type', 'collision_type',
       'incident_severity', 'property_damage', 'police_report_available',
       'insured_education_level', 'months_as_customer', 'age',
       'policy_deductable', 'policy_annual_premium', 'umbrella_limit',
       'capital-gains', 'capital-loss', 'incident_hour_of_the_day',
       'number_of_vehicles_involved', 'bodily_injuries', 'witnesses',
       'auto_year', 'total_claim_amount', 'fraud_reported'],
      dtype='object')


In [ ]:
X

,insured_sex,incident_type,collision_type,incident_severity,property_damage,police_report_available,insured_education_level,months_as_customer,age,policy_deductable,policy_annual_premium,umbrella_limit,capital-gains,capital-loss,incident_hour_of_the_day,number_of_vehicles_involved,bodily_injuries,witnesses,auto_year
0,FEMALE,Multi-vehicle Collision,Front Collision,Minor Damage,?,NO,MD,128,35,500,1366.60,0,0,0,10,3,2,1,2008
1,MALE,Single Vehicle Collision,Side Collision,Major Damage,?,?,Associate,116,34,500,1737.66,0,0,-24100,1,1,1,1,2001
2,FEMALE,Multi-vehicle Collision,Rear Collision,Major Damage,NO,YES,PhD,253,41,2000,1312.75,0,81300,0,10,3,2,2,2014
3,MALE,Single Vehicle Collision,Side Collision,Major Damage,YES,YES,College,45,37,1000,1114.23,0,0,0,1,1,0,1,2005
4,MALE,Vehicle Theft,?,Minor Damage,NO,NO,Masters,389,53,2000,791.47,0,0,0,6,1,1,2,2001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,FEMALE,Multi-vehicle Collision,Front Collision,Major Damage,YES,YES,JD,140,36,2000,979.26,0,0,-67000,2,4,0,2,1998
996,FEMALE,Multi-vehicle Collision,Front Collision,Major Damage,NO,?,High School,118,28,2000,1207.36,0,0,-57000,22,2,1,0,1997
997,FEMALE,Multi-vehicle Collision,Side Collision,Major Damage,YES,NO,Associate,115,31,1000,1051.67,0,0,0,18,3,0,3,2005
998,FEMALE,Multi-vehicle Collision,Side Collision,Total Loss,NO,NO,MD,312,47,1000,1212.07,0,66900,-51800,17,3,2,3,2000


# PROBLEMA DE CLASIFICACIÓN

In [ ]:

def evaluar_modelo(modelo, X, y, nombre_modelo="Modelo"):
    loo = LeaveOneOut()

    # Predicciones
    y_pred = cross_val_predict(modelo, X, y, cv=loo, n_jobs=-1)

    # Probabilidades (para ROC AUC)
    try:
        y_scores = cross_val_predict(modelo, X, y, cv=loo, method="predict_proba", n_jobs=-1)[:, 1]
        roc_auc = roc_auc_score(y, y_scores)
    except:
        roc_auc = np.nan

    pos_label = y.unique()[0]

    # Crear DataFrame con una fila
    return pd.DataFrame([{
        "Modelo": nombre_modelo,
        "Accuracy": accuracy_score(y, y_pred),
        "Precision": precision_score(y, y_pred, pos_label=pos_label, zero_division=0),
        "Recall": recall_score(y, y_pred, pos_label=pos_label, zero_division=0),
        "F1 Score": f1_score(y, y_pred, pos_label=pos_label, zero_division=0),
        "ROC AUC": roc_auc
    }])


modelos_y_parametros = {
    'LogisticRegression': (
        LogisticRegression(max_iter=3000, solver='liblinear'),
        {
            'model__penalty': ['l1', 'l2'],
            'model__C': [0.01, 0.1, 1, 10, 100, 1000, 10000, 1e-5],
            'model__class_weight': [None, 'balanced'],
            'model__fit_intercept': [True, False]
        }
    ),
    'DecisionTree': (
        DecisionTreeClassifier(),
        {
            'model__criterion': ['gini', 'entropy', 'log_loss'],
            'model__max_depth': [None, 3, 5, 10, 20, 30],
            'model__min_samples_split': [2, 5, 10, 20],
            'model__min_samples_leaf': [1, 2, 4, 6]
        }
    ),
    'KNN': (
        KNeighborsClassifier(),
        {
            'model__n_neighbors': [1, 3, 5, 7, 9, 11, 15, 21],
            'model__weights': ['uniform', 'distance'],
            'model__p': [1, 2, 3]
        }
    ),
    'SVM': (
        SVC(probability=True, kernel='rbf'),
        {
            'model__C': [0.01, 0.1, 1, 10, 100, 1000],
            'model__gamma': ['scale', 'auto', 0.001, 0.01, 0.1, 1]
        }
    ),
    'RandomForest': (
        RandomForestClassifier(),
        {
            'model__n_estimators': [50, 100, 200, 300, 500],
            'model__max_depth': [None, 5, 10, 20, 30],
            'model__min_samples_split': [2, 5, 10],
            'model__min_samples_leaf': [1, 2, 4],
            'model__max_features': ['sqrt', 'log2'],
            'model__class_weight': [None, 'balanced']
        }
    ),
    'GradientBoosting': (
        GradientBoostingClassifier(),
        {
            'model__n_estimators': [50, 100, 150, 200, 300],
            'model__learning_rate': [0.001, 0.01, 0.05, 0.1, 0.2],
            'model__max_depth': [2, 3, 4, 5, 6],
            'model__subsample': [0.6, 0.8, 1.0]
        }
    ),
    'AdaBoost': (
        AdaBoostClassifier(),
        {
            'model__n_estimators': [50, 100, 150, 200, 300],
            'model__learning_rate': [0.001, 0.01, 0.1, 0.5, 1.0, 2.0]
        }
    ),
    'MLPClassifier': (
        MLPClassifier(max_iter=500),
        {
            'model__hidden_layer_sizes': [(50,), (100,), (50, 50), (100, 50), (100, 100)],
            'model__activation': ['relu', 'tanh', 'logistic'],
            'model__solver': ['adam', 'lbfgs'],
            'model__alpha': [0.0001, 0.001, 0.01],
            'model__learning_rate': ['constant', 'invscaling', 'adaptive']
        }
    )
}


def correr_todos(X, y):
    resultados = []

    for nombre_modelo, (modelo, parametros) in modelos_y_parametros.items():
        print(f"🔍 Buscando mejores hiperparámetros para: {nombre_modelo}")

        pipeline = Pipeline([
            ('model', modelo)
        ])

        grid = GridSearchCV(pipeline, parametros, cv=3, n_jobs=-1)
        grid.fit(X, y)

        mejor_modelo = grid.best_estimator_
        mejores_parametros = grid.best_params_

        # Evaluar con LOO
        df_metricas = evaluar_modelo(mejor_modelo, X, y, nombre_modelo=nombre_modelo)
        df_metricas["Best Params"] = [mejores_parametros]

        resultados.append(df_metricas)

    # Concatenar todos los DataFrames
    df_resultados = pd.concat(resultados, ignore_index=True)
    df_resultados = df_resultados.sort_values(by="F1 Score", ascending=False)
    return df_resultados


In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

# Definir columnas categóricas y numéricas
cat_cols = X.columns[:7]

num_cols = X.columns[7:]

#Estandarizamos las columnas numéricas
X_estandarizada = StandardScaler().fit_transform(X[num_cols])
X_estandarizada = pd.DataFrame(X_estandarizada, columns=num_cols, index=X.index)
X[num_cols] = X_estandarizada

# Crear el preprocesador (solo OneHotEncoder)
preprocessor = ColumnTransformer([
    ('cat', OneHotEncoder(drop='first', sparse_output=False), cat_cols),
    ('num', 'passthrough', num_cols)  # deja las numéricas tal como están
])

# Ajustar y transformar
X_procesado = preprocessor.fit_transform(X)

In [ ]:
X

,insured_sex,incident_type,collision_type,incident_severity,property_damage,police_report_available,insured_education_level,months_as_customer,age,policy_deductable,policy_annual_premium,umbrella_limit,capital-gains,capital-loss,incident_hour_of_the_day,number_of_vehicles_involved,bodily_injuries,witnesses,auto_year
0,FEMALE,Multi-vehicle Collision,Front Collision,Minor Damage,?,NO,MD,-0.660150,-0.432150,-1.039966,0.451530,-0.479476,-0.901927,0.953851,-0.236618,1.140056,1.229693,-0.438431,0.481801
1,MALE,Single Vehicle Collision,Side Collision,Major Damage,?,?,Associate,-0.764448,-0.541611,-1.039966,1.971986,-0.479476,-0.901927,0.095895,-1.531974,-0.823865,0.009759,-0.438431,-0.682372
2,FEMALE,Multi-vehicle Collision,Rear Collision,Major Damage,NO,YES,PhD,0.426281,0.224613,1.412784,0.230875,-0.479476,2.016419,0.953851,-0.236618,1.140056,1.229693,0.461838,1.479664
3,MALE,Single Vehicle Collision,Side Collision,Major Damage,YES,YES,College,-1.381541,-0.213229,-0.222383,-0.582581,-0.479476,-0.901927,0.953851,-1.531974,-0.823865,-1.210174,-0.438431,-0.017130
4,MALE,Vehicle Theft,?,Minor Damage,NO,NO,Masters,1.608318,1.538139,1.412784,-1.905122,-0.479476,-0.901927,0.953851,-0.812332,-0.823865,0.009759,0.461838,-0.682372
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,FEMALE,Multi-vehicle Collision,Front Collision,Major Damage,YES,YES,JD,-0.555853,-0.322690,1.412784,-1.135634,-0.479476,-0.901927,-1.431336,-1.388046,2.122017,-1.210174,0.461838,-1.181303
996,FEMALE,Multi-vehicle Collision,Front Collision,Major Damage,NO,?,High School,-0.747065,-1.198374,1.412784,-0.200972,-0.479476,-0.901927,-1.075338,1.490523,0.158096,0.009759,-1.338700,-1.347613
997,FEMALE,Multi-vehicle Collision,Side Collision,Major Damage,YES,NO,Associate,-0.773139,-0.869992,-0.222383,-0.838927,-0.479476,-0.901927,0.953851,0.914809,1.140056,-1.210174,1.362107,-0.017130
998,FEMALE,Multi-vehicle Collision,Side Collision,Total Loss,NO,NO,MD,0.939076,0.881376,-0.222383,-0.181672,-0.479476,1.499516,-0.890219,0.770881,1.140056,1.229693,1.362107,-0.848682


In [ ]:
res = correr_todos(X_procesado, y_fraud)

🔍 Buscando mejores hiperparámetros para: LogisticRegression
🔍 Buscando mejores hiperparámetros para: DecisionTree
🔍 Buscando mejores hiperparámetros para: KNN
🔍 Buscando mejores hiperparámetros para: SVM
🔍 Buscando mejores hiperparámetros para: RandomForest
🔍 Buscando mejores hiperparámetros para: GradientBoosting
🔍 Buscando mejores hiperparámetros para: AdaBoost
🔍 Buscando mejores hiperparámetros para: MLPClassifier


In [ ]:
res

NameError: name 'res' is not defined

### para la continua claims_total

In [ ]:
def evaluar_modelo_continuas(modelo, X, y, nombre_modelo="Modelo"):
    loo = LeaveOneOut()

    # Predicciones
    y_pred = cross_val_predict(modelo, X, y, cv=loo, n_jobs=-1)

    # Calcular métricas de regresión
    mse = mean_squared_error(y, y_pred)
    RMSE = np.sqrt(mse)
    r2 = r2_score(y, y_pred)
    mape = np.mean(np.abs((y - y_pred) / y)) * 100  # MAPE en porcentaje

    # Crear DataFrame con una fila
    return pd.DataFrame([{
        "Modelo": nombre_modelo,
        "MSE": mse,
        "RMSE": RMSE,
        "MAPE": mape,
        "R2": r2
    }])



modelos_y_parametros_continua = {
    'PoissonRegressor': (
        PoissonRegressor(max_iter=1000),
        {
        'model__alpha': [0.0, 0.01, 0.1, 1.0],
        'model__fit_intercept': [True, False],
        'model__solver': ['lbfgs', 'newton-cholesky']
        }
    ),

    'GammaRegressor': (
        GammaRegressor(max_iter=1000),
    {
        'model__alpha': [0.0, 0.01, 0.1, 1.0],
        'model__fit_intercept': [True, False],
        'model__solver': ['lbfgs', 'newton-cholesky']
    }
    ),

    'DecisionTreeRegressor': (
        DecisionTreeRegressor(),
    {
        'model__criterion': ['squared_error', 'friedman_mse', 'absolute_error'],
        'model__max_depth': [None, 3, 5, 10, 20],
        'model__min_samples_split': [2, 5, 10],
        'model__min_samples_leaf': [1, 2, 4]
    }
    ),

    'KNeighborsRegressor': (
    KNeighborsRegressor(),
    {
        'model__n_neighbors': [3, 5, 7, 9],
        'model__weights': ['uniform', 'distance'],
        'model__p': [1, 2]
    }
    ),

    'SVR': (
        SVR(),
    {
        'model__kernel': ['linear', 'poly', 'rbf'],
        'model__C': [0.1, 1, 10],
        'model__epsilon': [0.01, 0.1, 0.2],
        'model__gamma': ['scale', 'auto']
    }
    ),

    'MLPRegressor': (
        MLPRegressor(max_iter=500),
    {
        'model__hidden_layer_sizes': [(50,), (100,), (50, 50), (100, 50), (100, 100)],
        'model__activation': ['relu', 'tanh'],
        'model__solver': ['adam', 'lbfgs'],
        'model__alpha': [0.0001, 0.001, 0.01],
        'model__learning_rate': ['constant', 'invscaling', 'adaptive']
    }
    )

}

def correr_todos_continuas(X, y):
    resultados = []

    for nombre_modelo, (modelo, parametros) in modelos_y_parametros_continua.items():
        print(f"🔍 Buscando mejores hiperparámetros para: {nombre_modelo}")

        pipeline = Pipeline([
            ('model', modelo)
        ])

        grid = GridSearchCV(pipeline, parametros, cv=3, n_jobs=-1)
        grid.fit(X, y)

        mejor_modelo = grid.best_estimator_
        mejores_parametros = grid.best_params_

        # Evaluar con LOO
        df_metricas = evaluar_modelo_continuas(mejor_modelo, X, y, nombre_modelo=nombre_modelo)
        df_metricas["Best Params"] = [mejores_parametros]

        resultados.append(df_metricas)

    # Concatenar todos los DataFrames
    df_resultados = pd.concat(resultados, ignore_index=True)
    df_resultados = df_resultados.sort_values(by="R2", ascending=False)
    return df_resultados

In [ ]:
res2 = correr_todos_continuas(X_procesado, y_claim)

🔍 Buscando mejores hiperparámetros para: PoissonRegressor
🔍 Buscando mejores hiperparámetros para: GammaRegressor
🔍 Buscando mejores hiperparámetros para: DecisionTreeRegressor


KeyboardInterrupt: 

In [ ]:
res2

,Modelo,MSE,MAPE,R2,Best Params,RMSE
0,PoissonRegressor,2.133738e+08,30.154342,0.693580,"{'model__alpha': 1.0, 'model__fit_intercept': ...",14607.319757
1,GammaRegressor,2.137144e+08,29.890367,0.693091,"{'model__alpha': 0.0, 'model__fit_intercept': ...",14618.975393
